In [1]:
from src.all_in_one import *
import os

# Set the pandas option
pd.set_option("future.no_silent_downcasting", True)

No password for tt_pt_FM_v04.11.xlsx


In [2]:
processed_data_path = path_manager.processed_data

In [3]:
# Creating a list of all rid across all datasets
rid = pd.DataFrame()  # An empty dataframe
if os.path.exists(processed_data_path) and os.path.isdir(processed_data_path):
    for file_name in os.listdir(processed_data_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(processed_data_path, file_name)
            df = pd.read_csv(file_path)
            df_rid = df["rid"]
            rid = pd.concat([rid, df_rid])
rid = rid.drop_duplicates(keep="first")
rid = rid.sort_values(by="rid", ascending=True).reset_index(drop=True)


In [31]:
# rid
# rid.to_clipboard(index=False)
# os.getcwd()

In [13]:
# It's better to consolidate all rid, firstnames, lastnames, sex, and age across all datasets first.
# because some of the referral id don't have these values from the screening.
# So, i needed to collect them from other services.

sub_id = pd.read_excel(
    r"notebooks/consolidate_datasets/sub_id.xlsx"
)  # It seems that I forget how I created "sub_id.xlsx"!
merged_df = sub_id.copy()  # Initialize merged_df to avoid referencing before assignment

for filename in os.listdir(processed_data_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(processed_data_path, filename)
        data = pd.read_csv(file_path)

        # Check if the needed columns exist in this dataset
        columns_needed = ["rid", "firstname", "lastname", "sex", "age"]
        common_columns = [col for col in columns_needed if col in data.columns]

        # Ensure 'rid' is present before merging
        if "rid" in common_columns:
            merged_df = merged_df.merge(
                data[common_columns],
                on="rid",
                how="left",
                suffixes=("", f"_from_{filename}"),
            )

# Optional: Consolidate columns, for example, firstname from different files
for col in ["firstname", "lastname", "sex", "age"]:
    columns_to_combine = [c for c in merged_df.columns if c.startswith(col)]
    merged_df[col] = (
        merged_df[columns_to_combine].bfill(axis=1).iloc[:, 0].infer_objects()
    )
    merged_df.drop(
        columns=columns_to_combine[1:], inplace=True
    )  # Drop extra columns after combining

In [14]:
df = merged_df.drop_duplicates(subset="rid")

In [29]:
df.shape
# df.to_clipboard(index=False)

(169, 5)

In [15]:
# Screening Dataset
scr_file = path_manager.get_data_file(Category.PS, PSFile.SCR)
scr_dataset = Dataset(config_file, scr_file.sheet)
scr = (
    get_df(scr_file.path, scr_file.sheet, config_file)
    .sort_values(by="sc_s1", ascending=True)
    .reset_index(drop=True)
)

In [18]:
# Get Screening basic session separated form pei_pre_as and re-screening
# columns_to_drop = ['scspi','nat', 'need_mhpss', 'need_trw', 'need_out_ref', 'need_tmh', 'need_pei', 'note','pei_pre_as', 'sc_re']
columns_to_drop = ["note", "pei_pre_as", "sc_re"]
scr_0 = (
    scr.drop(columns=columns_to_drop).dropna(subset=["sc_s1"]).sort_values(by="sc_s1")
)  # Screening basic session. Records = 705
scr_0 = scr_0.drop_duplicates(
    subset=["rid"], keep="first"
)  # There are 6 rid duplicated.
re_scr = scr[["rid", "sc_re"]].dropna()  # Screening re. Records = 41
pei_pre_as = scr[["rid", "pei_pre_as"]].dropna()  # PEI Assessment. Records = 36

In [25]:
pei_pre_as.shape
# re_scr.shape
# scr_0.shape

(67, 2)

Note: You need to rearrange the two columns to be in their positions.


In [26]:
consolidated_dataset_scr = pd.merge(rid, scr_0, on="rid", how="left")
consolidated_dataset_scr = pd.merge(
    consolidated_dataset_scr, re_scr, on="rid", how="left"
)
consolidated_dataset_scr = pd.merge(
    consolidated_dataset_scr, pei_pre_as, on="rid", how="left"
)

In [ ]:
# consolidated_dataset_scr.to_csv('consolidated_dataset.csv', index=False)

In [27]:
# pd.read_csv(r'notebooks/consolidate_datasets/jun24/consolidated_dataset.csv')